Importing libraries to use.

In [ ]:
!pip install deap

import array
import itertools
import math
import operator
import random

import numpy

from deap import base
from deap.benchmarks import movingpeaks
from deap import creator
from deap import tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 791.8 kB/s eta 0:00:00


We initializes the problem scenario and set up the necessary parameters.


The brown_ind function generates individuals with Brownian motion, a concept often used in the context of random motion.


DEAP entities (fitness and individuals) are created using the creator.create function.


The toolbox is set up with functions for creating individuals, populations, and performing selections and evaluations.

**ACTIVITIES**
*   Change SCENARIO_2 to SCENARIO_1 and observe the differences.
*   Change the dimension (NDIM) and observe the result.





In [ ]:
scenario = movingpeaks.SCENARIO_2

NDIM = 5
BOUNDS = [scenario["min_coord"], scenario["max_coord"]]

mpb = movingpeaks.MovingPeaks(dim=NDIM, **scenario)

def brown_ind(iclass, best, sigma):
    return iclass(random.gauss(x, sigma) for x in best)

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", array.array, typecode='d', fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, BOUNDS[0], BOUNDS[1])
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, NDIM)
toolbox.register("brownian_individual", brown_ind, creator.Individual, sigma=0.3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("select", random.sample, k=4)
toolbox.register("best", tools.selBest, k=1)
toolbox.register("evaluate", mpb)

We define the main function that will execute the differential evolution algorithm.

We set parameters like population size (NPOP), crossover rate (CR), and scaling factor (F).

The numbers regular and brownian represent the sizes of regular and Brownian subpopulations, respectively.

**ACITIVITIES:**


*   Change population size (NPOP) and observe the result.
*   Change crossover rate(CR) and scaling factor (F) and observe the difference.
*   Modify the stopping criteria: In the 'while' loop (while mpb.nevals < 5e5) students can change the stopping condition. For instance, they can decrease or increase the maximum number of evaluations to observe how it affects the algorithm's performance.




In [ ]:
def main(verbose=True):
    NPOP = 10   # Should be equal to the number of peaks
    CR = 0.6
    F = 0.4
    regular, brownian = 4, 2

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    logbook = tools.Logbook()
    logbook.header = "gen", "evals", "error", "offline_error", "avg", "max"

    # Initialize populations
    populations = [toolbox.population(n=regular + brownian) for _ in range(NPOP)]

    # Evaluate the individuals
    for idx, subpop in enumerate(populations):
        fitnesses = toolbox.map(toolbox.evaluate, subpop)
        for ind, fit in zip(subpop, fitnesses):
            ind.fitness.values = fit

    record = stats.compile(itertools.chain(*populations))
    logbook.record(gen=0, evals=mpb.nevals, error=mpb.currentError(),
                   offline_error=mpb.offlineError(), **record)
    if verbose:
        print(logbook.stream)

    g = 1
    while mpb.nevals < 5e5:
        # Detect a change and invalidate fitnesses if necessary
        bests = [toolbox.best(subpop)[0] for subpop in populations]
        if any(b.fitness.values != toolbox.evaluate(b) for b in bests):
            for individual in itertools.chain(*populations):
                del individual.fitness.values

        # Apply exclusion
        rexcl = (BOUNDS[1] - BOUNDS[0]) / (2 * NPOP**(1.0/NDIM))
        for i, j in itertools.combinations(range(NPOP), 2):
            if bests[i].fitness.valid and bests[j].fitness.valid:
                d = sum((bests[i][k] - bests[j][k])**2 for k in range(NDIM))
                d = math.sqrt(d)

                if d < rexcl:
                    if bests[i].fitness < bests[j].fitness:
                        k = i
                    else:
                        k = j

                    populations[k] = toolbox.population(n=regular + brownian)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in itertools.chain(*populations) if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        record = stats.compile(itertools.chain(*populations))
        logbook.record(gen=g, evals=mpb.nevals, error=mpb.currentError(),
                       offline_error=mpb.offlineError(), **record)
        if verbose:
            print(logbook.stream)

        # Evolve the sub-populations
        for idx, subpop in enumerate(populations):
            newpop = []
            xbest, = toolbox.best(subpop)
            # Apply regular DE to the first part of the population
            for individual in subpop[:regular]:
                x1, x2, x3, x4 = toolbox.select(subpop)
                offspring = toolbox.clone(individual)
                index = random.randrange(NDIM)
                for i, value in enumerate(individual):
                    if i == index or random.random() < CR:
                        offspring[i] = xbest[i] + F * (x1[i] + x2[i] - x3[i] - x4[i])
                offspring.fitness.values = toolbox.evaluate(offspring)
                if offspring.fitness >= individual.fitness:
                    newpop.append(offspring)
                else:
                    newpop.append(individual)

            # Apply Brownian to the last part of the population
            newpop.extend(toolbox.brownian_individual(xbest) for _ in range(brownian))

            # Evaluate the brownian individuals
            for individual in newpop[-brownian:]:
                individual.fitness.value = toolbox.evaluate(individual)

            # Replace the population
            populations[idx] = newpop

        g += 1

    return logbook

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
505	46837	1.27302 	3.51555      	10.8266   	68.4251 
506	46927	1.27302 	3.51124      	16.5476   	68.4251 
507	47017	1.27302 	3.50696      	19.6202   	68.4251 
508	47107	1.27302 	3.50269      	21.8321   	68.4251 
509	47197	1.27302 	3.49844      	24.054    	68.4251 
510	47287	1.27302 	3.49421      	26.4343   	68.4251 
511	47381	1.27302 	3.4898       	19.8167   	68.4251 
512	47475	1.27302 	3.48541      	19.2767   	68.4251 
513	47569	1.27302 	3.48104      	22.8032   	68.4251 
514	47659	1.27302 	3.47687      	26.8092   	68.4251 
515	47753	1.27302 	3.47253      	22.1625   	68.4251 
516	47847	1.27302 	3.46821      	19.2932   	68.4251 
517	47937	1.13666 	3.46392      	26.1005   	68.5615 
518	48027	1.13666 	3.45956      	28.4655   	68.5615 
519	48117	1.13666 	3.45521      	28.6149   	68.5615 
520	48211	1.13666 	3.45069      	23.5641   	68.5615 
521	48301	1.13666 	3.44638      	24.931    	68.5615 
522	48391	1.13666 	3.44208      	25.2814   	68.5

Source: [Dynamic DEAP GitHub](https://github.com/DEAP/deap/blob/master/examples/de/dynamic.py)